In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.base import TransformerMixin
import datetime

pd.set_option('display.max_columns', None)


In [77]:
initial_data_df = pd.read_csv('initial_data.csv',na_filter=False)

In [78]:
def normalize_column(df:pd.DataFrame,column:str,scaler:TransformerMixin):
    df[column] = scaler.fit_transform(np.array(df[column]).reshape(-1, 1))

In [79]:
def prep_df(df: pd.DataFrame):
    copy = df.copy()

    copy['is_medal'] = (copy['name'].str.contains('medal',case=False) | copy['description'].str.contains('medal',case=False)).astype('int')

    copy = copy.drop(columns=['name','description'], errors='ignore')

    copy['passed_time'] = copy['updated_time'] - copy['posted_time']

    copy['posted_year'] = copy['posted_time'].map(lambda ms:datetime.datetime.fromtimestamp(ms).year)
    copy['updated_year'] = copy['updated_time'].map(lambda ms:datetime.datetime.fromtimestamp(ms).year)

    copy['posted_month'] = copy['posted_time'].map(lambda ms:datetime.datetime.fromtimestamp(ms).month)
    copy['updated_month'] = copy['updated_time'].map(lambda ms:datetime.datetime.fromtimestamp(ms).month)

    copy = copy.drop(columns=['posted_time','updated_time'], errors='ignore')
    
    copy['media_count'] = copy['movie_count'] + copy['screenshot_count'] 

    submitter_value_counts = copy['submitter'].value_counts()
    copy['submitted_by_submitter'] =  copy['submitter'].map(lambda s:submitter_value_counts[s])

    copy = copy.drop(columns=['submitter'], errors='ignore')

    copy['v1'] = (copy['featured_collection_count'] / copy['favorites_count']).fillna(0)
    copy['v2'] = (copy['comments_count'] / (copy['discussions_count']+1)).fillna(0)
    copy['v3'] = (copy['visitors_count'] / (copy['discussions_count']+1)).fillna(0)
    copy['v4'] = (copy['visitors_count'] / (copy['comments_count']+1)).fillna(0)
    copy['v5'] = (copy['screenshot_count'] / (copy['media_count']+1)).fillna(0)

    copy['v6'] = (copy['class_spy'] == copy['class_pyro']).astype('int')
    copy['v7'] = (copy['class_engineer'] == copy['class_spy']).astype('int')
    copy['v8'] = (copy['item_slot_weapon'] == copy['item_slot_headgear']).astype('int')
      
    normalize_column(copy,'comments_count',StandardScaler())
    normalize_column(copy,'visitors_count',StandardScaler())
    normalize_column(copy,'discussions_count',StandardScaler())
    normalize_column(copy,'favorites_count',StandardScaler())
    normalize_column(copy,'submitted_by_submitter',StandardScaler())
    normalize_column(copy,'passed_time',StandardScaler())
    normalize_column(copy,'file_size_mb',StandardScaler())
    normalize_column(copy,'featured_collection_count',StandardScaler())
    normalize_column(copy,'change_notes',StandardScaler())
    normalize_column(copy,'creator_count',StandardScaler())

    print(f"shape {copy.shape}")
    print(f"non number {len(copy.select_dtypes(exclude=['number','category']).columns)}")
    print(f"na cells {copy.isna().sum().sum()}")

    return copy

In [80]:
prepped_df = prep_df(initial_data_df)

prepped_df.head()

shape (22100, 45)
non number 0
na cells 0


,discussions_count,comments_count,movie_count,screenshot_count,awards_count,link_count,file_size_mb,change_notes,certified_compatible,creator_count,featured_collection_count,visitors_count,favorites_count,accepted,class_count,item_slot_count,class_scout,class_soldier,class_pyro,class_demoman,class_heavy,class_engineer,class_medic,class_sniper,class_spy,item_slot_headgear,item_slot_misc,item_slot_taunt,item_slot_weapon,is_medal,passed_time,posted_year,updated_year,posted_month,updated_month,media_count,submitted_by_submitter,v1,v2,v3,v4,v5,v6,v7,v8
0,-0.299559,0.387851,0,6,0,0,0.026881,3.388062,1,0.044635,0.619064,-0.003128,0.170641,1,5,1,1,0,0,1,1,1,0,1,0,0,1,0,0,0,3.194280,2012,2016,9,3,6,2.741052,0.268692,154.00,5492.0,35.432258,0.857143,1,0,1
1,-0.299559,-0.119405,0,4,0,0,-0.363812,1.444371,1,0.044635,0.002978,-0.261479,-0.206975,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2.811226,2012,2015,9,10,4,2.741052,0.265896,40.00,2218.0,54.097561,0.800000,1,1,1
2,-0.299559,0.761618,0,2,0,0,-0.107814,-0.499321,0,1.583537,0.485132,0.502919,0.808887,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,-0.308865,2012,2012,9,9,2,0.607169,0.116414,238.00,11905.0,49.811715,0.666667,0,0,1
3,-0.299559,0.757168,0,8,0,0,-0.051529,0.796474,0,0.044635,0.404773,0.359304,0.499389,1,9,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,-0.281847,2012,2012,9,10,8,0.240555,0.140000,237.00,10085.0,42.373950,0.888889,1,1,1
4,0.320877,3.142160,0,0,0,0,-0.826900,-0.499321,0,-0.724816,-0.255957,2.183300,-0.312116,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-0.308865,2012,2012,2,2,0,-0.821686,0.166667,193.25,8300.0,42.894057,0.000000,1,1,1


In [81]:
prepped_df.to_csv('final_data.csv',index=False)